In [2]:
import pandas as pd
from pathlib import Path

from src import utils
from src.data import process_transcripts


ROOT_DIR = utils.get_project_root()
DATA_DIR = Path.joinpath(ROOT_DIR, 'data')
DATA_RAW_DIR = Path.joinpath(DATA_DIR, 'raw/cs-410')
INTERMEDATE_DATA_DIR = Path.joinpath(DATA_DIR, 'intermediate')

# Process Raw Transcript File

In [4]:
file_name = '04_week-4/02_week-4-lessons/01_lesson-4-1-probabilistic-retrieval-model-basic-idea.en.srt'
file_path = Path.joinpath(DATA_RAW_DIR, file_name)

transcript_segments = process_transcripts.process_transcript(file_path)

# print out first 5 segments
for s in transcript_segments[:5]:
    print(s)

Segment(id=0, beg=datetime.datetime(1900, 1, 1, 0, 0, 0, 86000), end=datetime.datetime(1900, 1, 1, 0, 0, 7, 516000), text='[SOUND] This')
Segment(id=1, beg=datetime.datetime(1900, 1, 1, 0, 0, 7, 516000), end=datetime.datetime(1900, 1, 1, 0, 0, 10, 282000), text='lecture is about the Probabilistic Retrieval Model.')
Segment(id=2, beg=datetime.datetime(1900, 1, 1, 0, 0, 10, 282000), end=datetime.datetime(1900, 1, 1, 0, 0, 11, 805000), text='In this lecture,')
Segment(id=3, beg=datetime.datetime(1900, 1, 1, 0, 0, 11, 805000), end=datetime.datetime(1900, 1, 1, 0, 0, 17, 806000), text="we're going to continue the discussion of the Text Retrieval Methods.")
Segment(id=4, beg=datetime.datetime(1900, 1, 1, 0, 0, 17, 806000), end=datetime.datetime(1900, 1, 1, 0, 0, 22, 942000), text="We're going to look at another kind of very different way to design ranking")
